In [1]:
import datetime
import time
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from utils import *
from scipy.signal import savgol_filter
import torch.nn.functional as F
import matplotlib.pyplot as plt
from models.timemil import *
from models.inceptiontime import *
from models.swin_transformer import *
from torchviz import make_dot
from sklearn.metrics import roc_curve, roc_auc_score,precision_recall_fscore_support,f1_score,accuracy_score,precision_score,recall_score,balanced_accuracy_score

In [16]:
class MultiView(nn.Module):
    def __init__(self,in_dim=137, out_dim=128, in_channels=64, out_channel=32, n_clip=3, clip_length=137, depth=2):
        super(MultiView, self).__init__()
        
        self.n_clip = n_clip
        self.ks = [3, 5, 17, 47]
        self.channels = [32, 64, 128, 256]
        
        self.convs = nn.ModuleList()
        self.batches = nn.ModuleList()
        self.activations = nn.ModuleList()
        
        for i in range(depth): 
            self.convs.append(nn.Conv1d(1 if i==0 else self.channels[i-1], self.channels[i], self.ks[i], padding='same', stride=1))
            self.batches.append(nn.BatchNorm1d(self.channels[i]))
            self.activations.append(nn.ReLU())

        # self.dropout = nn.Dropout(0.5)
            
            
    def forward(self, x):
        for conv, batch, activation in zip(self.convs, self.batches, self.activations):
            x = conv(x)
            x = batch(x)
            x = activation(x)

        return x[:,:,:-1]
            
        

In [17]:
class GlobleView(nn.Module):
    def __init__(self, in_dim=411, in_channels=64, out_dim=128, out_channel=32, depth=2):
        super(GlobleView, self).__init__()
        self.ks = [5, 17, 47]
        self.channels = [32, 64, 128, 256]

        self.convs = nn.ModuleList()
        self.batches = nn.ModuleList()
        self.activations = nn.ModuleList()

        for i in range(depth): 
            self.convs.append(nn.Conv1d(1 if i==0 else self.channels[i-1], self.channels[i], self.ks[i], padding='same', stride=1))
            self.batches.append(nn.BatchNorm1d(self.channels[i]))
            self.activations.append(nn.ReLU())
    
    def forward(self, x):
        for conv, batch, activation in zip(self.convs, self.batches, self.activations):
            x = activation(batch(conv(x)))
        return x[:,:,:-1]
        

In [23]:
class V3(nn.Module):
    def __init__(self, cross_in=64, in_dim=411, n_clip=3):
        super(V3, self).__init__()
        self.n_clip = n_clip
        self.rand_index = torch.randperm(n_clip)
        self.clip_length = in_dim // n_clip
        
        self.multiViews = nn.ModuleList()                       # [N,C,L] In: batch, channels, feature
        self.g_view = GlobleView(out_dim=128, out_channel=32)   # [S,N,E] out: batch, channels, feature [N, S, E] if batch_firese

        self.encodings = nn.ModuleList()
        # self.g_encoding = TransformerEncoder(input_dim=411, embed_dim=128, num_heads=8, num_layers=1, dropout=0.1)      # seq embedding
        # self.mv_encoder =TransformerEncoder(input_dim=137, embed_dim=128, num_heads=8, num_layers=1, dropout=0.1)
        
        self.g_encoding = SwinTransformer1D(num_classes=32, num_heads=[4, 8, 16, 32], patch_size=10, in_chans=64, embed_dim=64, drop_rate=0.0, drop_path_rate=0.5) # emberding=conv channels
        self.mv_encoder = SwinTransformer1D(num_classes=32, num_heads=[4, 8, 16, 32], patch_size=8, in_chans=64, embed_dim=128, drop_rate=0.0, drop_path_rate=0.5)
        
        
        self.crossAttentions = nn.ModuleList()
        
        for i in range(n_clip) : 
            self.multiViews.append(MultiView(out_dim=128, out_channel=32)) 
            self.crossAttentions.append(nn.MultiheadAttention(32, 8))

        self._fc2 = nn.Sequential(
            nn.Linear(32,32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, 4)) 
        
    def forward(self, x):
        outputs = []
        clips = torch.split(x, self.clip_length, dim=2)
        x = self.g_view(x)                                             # (batch_size, channels, seq_len)
        # print(x.shape)
        # 使用 Swin Transformer 进行全局编码
        x = self.g_encoding(x)    # (batch_size, channels, seq_len)
        
        for i in range(self.n_clip):
            multiview = self.multiViews[i](clips[i])
            
            # print(multiview.shape)
            
            multiview = self.mv_encoder(multiview)  # Swin transformer block
            attn_output, _ = self.crossAttentions[i](x, multiview, multiview)
            outputs.append(attn_output)

        x = self._fc2(x)
        return x

model = V3()
model
# 创建随机输入
input_tensor = torch.randn(64, 1, 411)

# 模型前向传播
output = model(input_tensor)
output.shape

torch.Size([64, 4])

In [5]:
data = pd.read_csv('data_multiClass.csv')
data = data.drop('Unnamed: 0', axis=1)

In [6]:
# 特征和标签
X = data.drop('category', axis=1).values
y = data['category'].values
print(X.shape)
print(y.shape)

(2566, 411)
(2566,)


In [7]:
#标准化特征
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(1/3), random_state=42)

# 将数据转换为卷积神经网络输入格式 (samples, timesteps, features)
# 将数据转换为卷积神经网络输入格式 (samples, channels, sequence_length)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(1796, 1, 411)
(1796,)
(513, 1, 411)
(513,)
(257, 1, 411)
(257,)


In [8]:
# 将数据转换为PyTorch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor  = F.one_hot(torch.tensor(y_train)).float()
y_val_tensor = F.one_hot(torch.tensor(y_val)).float()
y_test_tensor = F.one_hot(torch.tensor(y_test)).float()


# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
X_train_tensor.shape

torch.Size([1796, 1, 411])

In [9]:
def val(model, val_dataloader):
    val_labels = []
    val_predictions = []
    with torch.no_grad():
        for i, (inputs, label) in enumerate(val_dataloader, 0):
            # images = images.to(device).half() # uncomment for half precision model
            inputs = inputs.cuda()
            label = label.cuda()
            outputs = model(inputs)

            val_labels.extend([label.cpu().numpy()])
            val_predictions.extend([torch.sigmoid(outputs).cpu().numpy()])
    
    val_labels = np.vstack(val_labels)
    val_predictions = np.vstack(val_predictions)

    val_predictions_prob = np.exp(val_predictions)/np.sum(np.exp(val_predictions),axis=1,keepdims=True)
    
    
    val_predictions = np.argmax(val_predictions,axis=1)
    val_labels = np.argmax(val_labels,axis=1)
    avg_score = accuracy_score(val_labels,val_predictions)
    print('Accuracy of the network on the test dataset: {}'.format(avg_score))
    return avg_score

In [10]:
def train(model, criterion, optimizer, scheduler, n_epochs=5):
    best = 0
    train_step = 0
    losses = []

    # 定义自定义日志目录名称
    log_dir_name = "V3_2_multiClass"   # "V3_multiClass" 'V3_2_binary'
    log_dir = f"runs/{log_dir_name}_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    
    # 初始化TensorBoard
    writer = SummaryWriter(log_dir=log_dir)
    print(f"tensorboard --logdir={log_dir}")

    model.train()
    for epoch in range(n_epochs):
        since = time.time()
        train_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, targets = data
            
            inputs = inputs.cuda()
            targets = targets.cuda()
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, targets)
            loss.backward()

            # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()

            train_loss += loss.item()
            writer.add_scalar("train loss", loss.item(), train_step)
            train_step += 1
        
        epoch_duration = time.time() - since
        epoch_loss = train_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, duration: {epoch_duration:.2f}, loss: {epoch_loss:.6f}")

        losses.append(epoch_loss)

        # 验证模型
        model.eval()
        val_acc = val(model, val_loader)
        writer.add_scalar("test acc", val_acc, epoch + 1)
        if best < val_acc:
            best = val_acc
            torch.save(model, "V3_2_multiClass.pth")

        model.train()
        #scheduler.step() # test_acc
    
    print('Finished Training')
    return model, losses

In [19]:
# model_ft = WaveLength(c_in=X.shape[1], c_out=6, nf=[47, 47, 47, 47])
model_ft = V3().cuda()



#lrscheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, epochs=epochs, steps_per_epoch=len(train_dataloader))
# lrscheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=20, eta_min=1e-9)

# lrscheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# parms_1x = [value for name, value in model.named_parameters()
#             if name not in ["fc.weight", "fc.bias"]]
# parms_10x = [value for name, value in model.named_parameters()
#             if name in ["fc.weight", "fc.bias"]]
# optimizer = optim.AdamW([{"params": parms_1x},
#                         {"params": parms_10x, 'lr': learning_rate * 10}], lr=learning_rate)


learning_rate = 1e-5  # 2e-5
epochs = 300  #200

criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.AdamW(model_ft.parameters(), lr=learning_rate, weight_decay=learning_rate)
# lrscheduler = optim.lr_scheduler.OneCycleLR(optimizer,max_lr=learning_rate,pct_start=0.5,total_steps=150,div_factor=10,final_div_factor=10)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)
# 打印模型的每一层参数量
total_params = 0
for name, param in model_ft.named_parameters():
    if param.requires_grad:
        # print(f"{name}: {param.numel()}")
        total_params += param.numel()

print(f"Total number of trainable parameters: {total_params}")
model_ft

Total number of trainable parameters: 59634324


V3(
  (multiViews): ModuleList(
    (0-2): 3 x MultiView(
      (convs): ModuleList(
        (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=same)
        (1): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=same)
      )
      (batches): ModuleList(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (activations): ModuleList(
        (0-1): 2 x ReLU()
      )
    )
  )
  (g_view): GlobleView(
    (convs): ModuleList(
      (0): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=same)
      (1): Conv1d(32, 64, kernel_size=(17,), stride=(1,), padding=same)
    )
    (batches): ModuleList(
      (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (activations): ModuleList(
      (0-1): 2 x Re

In [20]:
model_ft, training_losses = train(model_ft, criterion, optimizer, lrscheduler, n_epochs=epochs)

tensorboard --logdir=runs/V3_2_multiClass_20240613-172153


/root/miniconda3/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1, duration: 2.51, loss: 0.662008
Accuracy of the network on the test dataset: 0.2573099415204678
Epoch 2, duration: 2.34, loss: 0.604497
Accuracy of the network on the test dataset: 0.28460038986354774
Epoch 3, duration: 2.35, loss: 0.596603
Accuracy of the network on the test dataset: 0.2787524366471735
Epoch 4, duration: 2.33, loss: 0.594657
Accuracy of the network on the test dataset: 0.30409356725146197
Epoch 5, duration: 2.35, loss: 0.593662
Accuracy of the network on the test dataset: 0.2884990253411306
Epoch 6, duration: 2.33, loss: 0.589337
Accuracy of the network on the test dataset: 0.29434697855750486
Epoch 7, duration: 2.33, loss: 0.586234
Accuracy of the network on the test dataset: 0.26705653021442494
Epoch 8, duration: 2.39, loss: 0.586284
Accuracy of the network on the test dataset: 0.30214424951267055
Epoch 9, duration: 2.34, loss: 0.584529
Accuracy of the network on the test dataset: 0.3001949317738791
Epoch 10, duration: 2.35, loss: 0.579054
Accuracy of the ne

In [24]:
model = torch.load('V3_2_multiClass.pth')
model.cuda()
model.eval()  # 切换模型到评估模式z
print("Model loaded successfully.")

Model loaded successfully.


In [25]:
test_labels = []
test_predictions = []
with torch.no_grad():
    for i, (inputs, label) in enumerate(test_loader, 0):
        # images = images.to(device).half() # uncomment for half precision model
        inputs = inputs.cuda()
        label = label.cuda()
        outputs = model(inputs)

        test_labels.extend([label.cpu().numpy()])
        test_predictions.extend([torch.sigmoid(outputs).cpu().numpy()])

test_labels = np.vstack(test_labels)
test_predictions = np.vstack(test_predictions)

test_predictions_prob = np.exp(test_predictions)/np.sum(np.exp(test_predictions),axis=1,keepdims=True)


test_predictions = np.argmax(test_predictions,axis=1)
test_labels = np.argmax(test_labels,axis=1)

avg_score = accuracy_score(test_labels, test_predictions)
p_marco = precision_score(test_labels,test_predictions,average='macro')
r_marco = recall_score(test_labels,test_predictions,average='macro')
f1_marco = f1_score(test_labels,test_predictions,average='macro')
print("Average accuracy: {:.4f}".format(avg_score))
print("f1_marco: {:.4f}".format(f1_marco))
print("p_marco: {:.4f}".format(p_marco))
print("r_marco: {:.4f}".format(r_marco))

Average accuracy: 0.8794
f1_marco: 0.8804
p_marco: 0.8817
r_marco: 0.8820


In [ ]:
#[ 1 1 2 2 3 3 4 4 5 5 0 0]
# [1 0 2 2 3 5 4 5 5 5 0 0]